In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from tqdm import tqdm
import sys
import matplotlib as mpl

#insert path
sys.path.insert(0, '../methods/')

sys.modules.pop('generate_syn_data', None)
from generate_syn_data import *

sys.modules.pop('ARWQE', None)
from ARWQE import *

sys.modules.pop('plots', None)
from plots import *

In [ ]:
import numpy as np

#given beta, compute alpha sequence
def pinball(u, alpha):
    return alpha * u - np.minimum(u, 0)

def conformal_adapt_stable(betas, alpha, gammas, sigma=1/1000, eta=2.72):
    T = len(betas)
    k = len(gammas)
    alpha_seq = np.full(T, alpha)
    err_seq_adapt = np.zeros(T)
    err_seq_fixed = np.zeros(T)
    gamma_seq = np.zeros(T)
    mean_alpha_seq = np.zeros(T)
    mean_err_seq = np.zeros(T)
    mean_gammas = np.zeros(T)
    
    expert_alphas = np.full(k, alpha)
    expert_ws = np.ones(k)
    cur_expert = np.random.choice(k)
    expert_cumulative_losses = np.zeros(k)
    expert_probs = np.full(k, 1/k)
    
    for t in range(T):
        alphat = expert_alphas[cur_expert]
        alpha_seq[t] = alphat
        err_seq_adapt[t] = float(alphat > betas[t])
        err_seq_fixed[t] = float(alpha > betas[t])
        gamma_seq[t] = gammas[cur_expert]
        mean_alpha_seq[t] = np.sum(expert_probs * expert_alphas)
        mean_err_seq[t] = float(mean_alpha_seq[t] > betas[t])
        mean_gammas[t] = np.sum(expert_probs * gammas)
        
        expert_losses = pinball(betas[t] - expert_alphas, alpha)
        expert_alphas += gammas * (alpha - (expert_alphas > betas[t]).astype(float))
        
        if eta < np.inf:
            expert_bar_ws = expert_ws * np.exp(-eta * expert_losses)
            expert_next_ws = (1 - sigma) * expert_bar_ws / np.sum(expert_bar_ws) + sigma / k
            expert_probs = expert_next_ws / np.sum(expert_next_ws)
            cur_expert = np.random.choice(k, p=expert_probs)
            expert_ws = expert_next_ws
        else:
            expert_cumulative_losses += expert_losses
            cur_expert = np.argmin(expert_cumulative_losses)
            
    return {
        "alpha_seq": alpha_seq,
        "err_seq_adapt": err_seq_adapt,
        "err_seq_fixed": err_seq_fixed,
        "gamma_seq": gamma_seq,
        "mean_alpha_seq": mean_alpha_seq,
        "mean_err_seq": mean_err_seq,
        "mean_gammas": mean_gammas
    }

# Example usage
betas = np.random.rand(50)
alpha = 0.05
#gammas = np.random.rand(5) * 0.01
gammas = np.array([0.001,0.002,0.004,0.008,0.0160,0.032,0.064,0.128])


results = conformal_adapt_stable(betas, alpha, gammas)
for key, val in results.items():
    print(key, "\n", val)

In [4]:
#given conformity scores, find beta
#from arch import arch_model
import numpy as np

def find_beta(recent_scores, cur_score, epsilon=0.001):
    top = 1
    bot = 0
    while top - bot > epsilon:
        mid = (top + bot) / 2
        if np.quantile(recent_scores, 1 - mid) > cur_score:
            bot = mid
        else:
            top = mid
    return (top + bot) / 2

def garch_conformal_forecasting_compute_betas(scores, lookback=float('inf'), epsilon=0.001):
    T = len(scores)
    beta_seq = np.zeros(T - 1)

    for t in range(1, T):
        if lookback == float('inf'):
            recent_scores = scores[:t]
        else:
            recent_scores = scores[max(0, t - lookback):t]

        beta_seq[t - 1] = find_beta(recent_scores, scores[t], epsilon)

        if (t + 1) % 1000 == 0:
            print(f"Done {t + 1} steps")

    return beta_seq

# Example usage
returns = np.random.randn(10000)  # Simulated returns
#scores, sigma_seq = garch_conformal_forecasting_compute_scores(returns)
scores = np.random.randn(100)
betas = garch_conformal_forecasting_compute_betas(scores)
print(betas.shape)


(99,)


In [7]:
alpha =0.1; delta = 0.1; gamma=1

num_periods = 1000; n=num_periods
num_trials = 2

#NOTE: uncomment to change shift patterns
#beta_1 = 2 * np.sin(np.linspace(0, 5*np.pi, num_periods)) + 5
#beta_1 = np.ones(num_periods)
beta_1 = generate_true_means(num_periods-1, 2)
beta_2 = np.ones(num_periods)
beta = np.column_stack((beta_1, beta_2))
variance_y = 1; meanX = 1

np.random.seed(6)

#B_arr is for val set
B_arr = np.random.randint(low=2, high=3, size=num_periods)
B_arr_starts = np.arange(num_periods)
B_arr_ends = np.cumsum(B_arr) - 1
B_arr_tr = B_arr * 2

np.random.seed(6)
X_tr, y_tr = generate_linreg_data(meanX, B_arr_tr, beta, variance_y)
X_val, y_val = generate_linreg_data(meanX, B_arr, beta, variance_y)
reg, S = fit_and_get_scores(X_tr, y_tr, X_val, y_val)

#take the scores from the past k periods
k = 10
S = S[B_arr_starts[n-k]:]

#compute betas



